In [1]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from functools import reduce
from matplotlib.tri import Triangulation

In [2]:
from k3d.platonic import PlatonicSolid
from itertools import product

class Box(PlatonicSolid):
    def __init__(self, origin=[0, 0, 0], size=[1,1,1], **kwargs):
        origin = np.array(origin, dtype=np.float32)
        size = np.array(size, dtype=np.float32)
        
        if origin.shape == (3,):
            cube_vertices = np.array(list(product([size[0], -size[0]], [size[1], -size[1]], [size[2], -size[2]])), np.float32)*0.5
            cube_vertices = np.float32(cube_vertices + origin)

            self.vertices = cube_vertices
            self.indices = [0, 1, 2, 1, 2, 3, 0, 1, 4, 1, 4, 5, 1, 3, 5, 3, 5, 7, 0, 2, 4, 2, 4, 6, 2, 3, 7, 2, 6, 7, 4,
                            5, 6, 5, 6, 7]

        else:
            raise TypeError('Origin attribute should have 3 coordinates.')

In [3]:
def getrot(a,b):
    if np.allclose(a,b):
        return np.identity(3)
    else:
        v = np.cross(a, b)
        s = np.linalg.norm(v)
        c = np.dot(a,b)
        vx = np.array([[0,-v[2], v[1]],[v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return np.identity(3) + vx + vx@vx*(1/(1+c))#(1-c)/s**2

def pad3to4(mat):
    m = np.identity(4)
    m[0:3,0:3] = mat
    return m

In [4]:
def norm(v):
    v = np.array(v)
    return v/np.linalg.norm(v)

In [5]:
def vec2rot(a, b):
    if np.allclose(a,b):
        return [0, 1, 0, 0]
    else:
        ez = norm(a)
        n = norm(b)
        ax = np.cross(ez,n)
        ang = np.arccos(ez@n)
        return [ang, ax[0], ax[1], ax[2]]

In [6]:
class Mirror:
    def __init__(self, p=(0,0,0), n=(0,0,1), diam=1.):
        self.p = np.array(p)
        self.n = np.array(norm(n))
        self.R = diam/2
    
    def plot_outline(self, npts=30, **kwargs):
        e = norm([1,1,1])
        na = norm(np.cross(self.n, self.n+e))
        nb = norm(np.cross(self.n, na))

        phi = np.linspace(0, 2*np.pi, npts, endpoint=True)
        vecs = self.p + self.R*np.stack([np.sin(p)*na + np.cos(p)*nb for p in phi], axis=0)
        return k3d.line(vecs, **kwargs)
    
    def plot(self, n_radii = 5, n_angles = 10, **kwargs):
        radii = np.linspace(0., 1, n_radii)
        x, y = [0], [0]
        for r in radii[1:]:
            x.extend([r*np.cos(p) for p in np.linspace(0,2*np.pi,int(2*np.pi*r*n_angles))])
            y.extend([r*np.sin(p) for p in np.linspace(0,2*np.pi,int(2*np.pi*r*n_angles))])
        x = np.array(x)*self.R
        y = np.array(y)*self.R

        z = np.zeros_like(x)
        indices = Triangulation(x,y).triangles.astype(np.uint32)
        mesh = k3d.mesh(np.vstack([x,y,z]).T, indices, **kwargs)
        mesh.transform.rotation = vec2rot(norm([0,0,1]), self.n)
        mesh.transform.translation = self.p
        return mesh
    
    def intersect_all(self, ray, return_msk=False):
        #shape of Ray [2xNrays]
        r, s = ray
        sn = s@self.n

        t = ((self.p - r)@self.n)/sn
        msk = np.abs(sn)>np.finfo(np.float32).eps
        x = r[msk] + t[msk,None]*s[msk]
        if return_msk:
            return x, msk
        else:
            return x
    
    def intersect(self, ray, return_msk=False):
        q, msk = self.intersect_all(ray, return_msk=True)
        d = np.linalg.norm(q - self.p, axis=1)
        msk = np.logical_and(msk, (d<=self.R))
        if return_msk:
            return q[msk], msk
        else:
            return q[msk]
    
    def propagate(self, ray, indices=None, aperture=True):
        _, s = ray
        if indices is None:
            indices = np.arange(ray.shape[1])
            
        if aperture:
            q, msk = self.intersect(ray, return_msk=True)
        else:
            q, msk  = self.intersect_all(ray, return_msk=True)
        s = s[msk]
        indices = indices[msk]
        
        sp = s - 2*np.dot(s, self.n)[:,None]*self.n[None,:]
        return np.stack([q, sp], axis=0), indices

In [7]:
class Screen(Mirror):
    def __init__(self, p=(0,0,0), n=(0,0,1), diam=1.):
        super().__init__(p, n, diam)
        
    def propagate(self, ray, indices=None, aperture=True):
        _, s = ray
        if indices is None:
            indices = np.arange(ray.shape[1])
            
        if aperture:
            q, msk = self.intersect(ray, return_msk=True)
        else:
            q, msk  = self.intersect_all(ray, return_msk=True)
        s = s[msk]
        indices = indices[msk]
        return np.stack([q, s], axis=0), indices

In [8]:
class Glass(Mirror):
    def __init__(self, p=(0,0,0), n=(0,0,1), diam=1., n1=1., n2=1.):
        super().__init__(p, n, diam)
        self.n1 = n1
        self.n2 = n2
        self.r = n1/n2
        
    def propagate(self, ray, indices=None, aperture=True):
        #Propagate rays by transmission through  an index chane, Snells law
        if indices is None:
            indices = np.arange(ray.shape[1])
        
        _, s = ray
        if aperture:
            q, msk = self.intersect(ray, return_msk=True)
        else:
            q, msk  = self.intersect_all(ray, return_msk=True)
        s = s[msk]
        indices = indices[msk]
        
        c = -np.dot(s, self.n)
        dis = 1 - self.r**2*(1 - c**2)
        #prevent total internal reflection
        msk = (dis>=0.)
        sp = self.r*s + (self.r*c - np.sqrt(dis))[:,None]*self.n[None,:]
        indices = indices[msk]
        
        return np.stack([q[msk], sp[msk]], axis=0), indices        

In [9]:
def dbg_shape(lst):
    for l in lst:
        print(l.shape)

In [10]:
class CurvedMirror:
    def __init__(self, p=(0,0,0), n=(0,0,1), R=1., curv='CC', diam=25.4):
        self.p = np.array(p)
        self.n = np.array(norm(n))
        self.R = R
        self.rapt = diam/2.
        if curv=='CC':
            self.cp = self.p + self.R*self.n
        elif curv=='CX':
            self.cp = self.p - self.R*self.n
        else:
            raise ValueError("Mirror type {} unknown! Curv has to be CC or CX".format(curv))
        self.curv = curv
        
    def plot(self, n_radii = 10, n_angles = 10, **kwargs):
        radii = np.linspace(0., 1, n_radii)
        x, y = [0], [0]
        for r in radii[1:]:
            x.extend([r*np.cos(p) for p in np.linspace(0,2*np.pi,int(2*np.pi*r*n_angles))])
            y.extend([r*np.sin(p) for p in np.linspace(0,2*np.pi,int(2*np.pi*r*n_angles))])
        x = np.array(x)*self.rapt
        y = np.array(y)*self.rapt

        z = self.R-np.sqrt(self.R**2-x**2-y**2)
        if self.curv=='CX':
            z = -z

        
        indices = Triangulation(x,y).triangles.astype(np.uint32)
        mesh = k3d.mesh(np.vstack([x,y,z]).T, indices, **kwargs)
        mesh.transform.rotation = vec2rot(norm([0,0,1]), self.n)
        mesh.transform.translation = self.p
        return mesh
    
    def intersect_all(self, ray, return_msk=False):
        #shape of Ray [2xNrays]
        r, s = ray
        d = self.cp - r
        ds = np.einsum("ji,ji->j",d,s)
        
        dis = ds**2 + self.R**2 - np.einsum("ji,ji->j",d,d)
        msk = dis>=0
        t12 = np.stack([ds+np.sqrt(dis), ds-np.sqrt(dis)], axis=0) #[2xNrays]
        #find the right intersection closer to the origin!
        x12 = r[None,msk] + t12[:,msk,None]*s[None,msk] #[2xNraysx3]
        dist12 = np.linalg.norm(x12-self.p, axis=2) #[2xNrays]
        which = np.argmin(dist12, axis=0)
        x = np.stack([x12[w,i,:] for i, w in enumerate(which)]) #TODO: find vector expression
        
        if return_msk:
            return x, msk
        else:
            return x
        
    def intersect(self, ray, return_msk=False):
        xb = np.sqrt(self.R**2 - self.rapt**2)
        x = self.R - xb
        dst = np.sqrt(self.rapt**2+x**2)
        
        q, msk = self.intersect_all(ray, return_msk=True)
        d = np.linalg.norm(q - self.p, axis=1) #TODO not the right distance! 
        msk = np.logical_and(msk, (d<=dst))
        if return_msk:
            return q[msk], msk
        else:
            return q[msk]
        
    def propagate(self, ray, indices=None, aperture=True):
        if indices is None:
            indices = np.arange(ray.shape[1])
            
        _, s = ray
        if aperture:
            q, msk = self.intersect(ray, return_msk=True)
        else:
            q, msk  = self.intersect_all(ray, return_msk=True)
        s = s[msk]
        indices = indices[msk]
        #find normal vectors
        n = self.cp - q
        nn = np.linalg.norm(n, axis=1)
        n *= 1./nn[:,None]
        
        #sp = s - 2*np.dot(s, n)[:,None]*n[None,:]
        sp = s - 2*np.einsum("ji,ji->j",s,n)[:,None]*n
        
        return np.stack([q, sp], axis=0), indices

In [11]:
class CurvedGlass(CurvedMirror):
    def __init__(self, p=(0,0,0), n=(0,0,1), R=1., curv='CC', diam=25.4, n1=1., n2=1.):
        super().__init__(p, n, R, curv, diam)
        self.n1 = n1
        self.n2 = n2
        self.r = n1/n2
        
    def propagate(self, ray, indices=None, aperture=True):
        #Propagate rays by transmission through  an index change, Snells law
        _, s = ray
        if indices is None:
            indices = np.arange(ray.shape[1])
            
        if aperture:
            q, msk = self.intersect(ray, return_msk=True)
        else:
            q, msk  = self.intersect_all(ray, return_msk=True)
        s = s[msk]
        indices = indices[msk]
        #find normal vectors
        n = self.cp - q
        nn = np.linalg.norm(n, axis=1)
        n *= 1./nn[:,None]
        if self.curv=='CX':
            n = -n
        #c = -np.dot(s, self.n)
        #dis = 1 - self.r**2*(1 - c**2)
        #msk = (dis>=0.)
        #sp = self.r*s + (self.r*c - np.sqrt(dis))[:,None]*self.n[None,:]
        c = -np.einsum("ji,ji->j",s,n)
        dis = 1 - self.r**2*(1 - c**2)
        msk = (dis>=0.)
        sp = self.r*s + (self.r*c - np.sqrt(dis))[:,None]*n
        indices = indices[msk]
        
        return np.stack([q[msk], sp[msk]], axis=0), indices        

In [12]:
mir = CurvedGlass(p=(0,0,10), n=norm((0,0.1,-1)), R=8., diam=5, curv='CC', n2=1.5)

In [ ]:
mir = CurvedMirror(p=(0,0,10), n=norm((0,0.1,-1)), R=20., diam=5, curv='CC')

In [ ]:
plot = k3d.plot(camera_auto_fit=True, antialias=True)

plot_rays(rays, plot)
plot += mir.plot(opacity=0.3)
xi = mir.intersect(rays)
pts = k3d.points(xi, point_size=0.1, color=0x00ff00)
plot += pts
rays2, ind2 = mir.propagate(rays)
plot_rays(rays2, plot)
plot.display()

In [ ]:
Nray = 12
org = np.stack([np.array([0,0,0]) for _ in range(Nray)], axis=0)
s = np.stack([norm([3,np.random.rand()-.5,np.random.rand()-.5]) for _ in range(Nray)], axis=0)
rays = np.stack([org, s], axis=0)
rays.shape

In [13]:
def plot_rays(rays, plot, length=10.):
    s = k3d.vectors(origins=rays[0,...], vectors=rays[1,...], colors=[(0xff0000,)*rays.shape[1]*2], head_size=2.)
    plot += s
    ls = [k3d.line([rays[0,i,:], rays[0,i,:]+15*rays[1,i,:]]) for i in range(rays.shape[1])]
    for l in ls:
        plot += l

In [14]:
def ray_bundle(p=(0,0,0), n=(0,0,1), n_radii=10, n_angles=2, R=1., divergence=0.):
    radii = np.linspace(0., 1, n_radii)
    x, s = [[0,0,0],], [[0,0,1],]
    for r in radii[1:]:
        for p in np.linspace(0,2*np.pi,int(2*np.pi*r*n_angles)):
            x.append( [R*r*np.cos(p), R*r*np.sin(p), 0])
            s.append(norm([divergence*r*np.cos(p), divergence*r*np.sin(p), 1]))
       
    rays = np.stack([np.array(x), np.array(s)], axis=0)
    ez = np.array([0,0,1])
    nn = norm(n)
    R = getrot(ez, nn)
    rays2 = np.einsum("abi,ij->abj", rays, R)
    #rays2[0,:,:] += p
    return rays2

In [ ]:
rays = ray_bundle(n_radii=5,n_angles=2)

In [ ]:
ap1 = Mirror(p=(10,0,0), n=(1,0,0), diam=2.)

In [ ]:
ap1 = Glass(p=(10,0,0), n=(-1,0,0), diam=2., n2=1.5)

In [ ]:
plot = k3d.plot(camera_auto_fit=True, antialias=True)

#bbox = k3d.platonic.Cube(origin=(10,0,0), size=2)
#plot += k3d.mesh(bbox.vertices, bbox.indices, wireframe=True)

plot_rays(rays, plot)
plot += ap1.plot()
xi = ap1.intersect(rays)
pts = k3d.points(xi, point_size=0.2)
plot += pts
rays2 = ap1.propagate(rays)
plot_rays(rays2, plot)
plot.display()

In [15]:
def geometry(mir):
    
    def norm(arr, axis=-1):
        norm = np.sqrt(np.sum(arr**2, axis=-1))
        norm = np.expand_dims(norm, axis)
        return np.divide(arr, norm, where=(norm!=0))
    
    Nm = len(mir)
    M = mir[:,None,:]-mir[None,:,:]
    m = norm(M)
    
    n = norm(np.array([np.cross(m[j,j-1],m[j,(j+1)%Nm]) for j in range(Nm)]))
    refl = -norm(np.array([0.5*(m[j,j-1]+m[j,(j+1)%Nm]) for j in range(Nm)])) #vectors normal to reflecting mirrors
    angles = np.array([0.5*np.arccos(np.dot(m[j,j-1],m[j,(j+1)%Nm])) for j in range(Nm)])
    xin = n
    xout = n
    yin = norm(np.array([np.cross(n[j],m[j,j-1]) for j in range(Nm)]))
    yout = norm(np.array([np.cross(n[j],m[j,(j+1)%Nm]) for j in range(Nm)]))
    R = np.stack([np.array([[xout[i]@xin[(i+1)%Nm], yout[i]@xin[(i+1)%Nm]],\
                            [xout[i]@yin[(i+1)%Nm], yout[i]@yin[(i+1)%Nm]]]) for i in range(Nm)], axis=0)
    
    Ls = [np.linalg.norm(M[j-1,j]) for j in range(Nm)]
    Lrt = sum(Ls)

    return {'mir': mir, 'n': n, 'refl': refl, 'xin': xin, 'xout': xout, 'yin': yin, 'yout': yout, 'Ls': Ls, 'Lrt': Lrt}

In [16]:
def plot_geometry(geom, **kwargs):
    mir, n, refl, yin, yout = geom['mir'], geom['n'], geom['refl'], geom['yin'], geom['yout']
    Nm = len(mir)
    plot = k3d.plot(camera_auto_fit=True, antialias=True)

    col=0xff0000
    pf = 1.
    plt_line = k3d.line(pf*mir, shader='mesh', width=0.5, color=col)
    plt_line2 = k3d.line(pf*mir[(-1,0),...], shader='mesh', width=0.5, color=col)
    plot += plt_line
    plot += plt_line2
    plot += k3d.vectors(origins=pf*mir, vectors=n*2, use_head=True, head_size=3.)#Normals = xIn = xOut
    plot += k3d.vectors(origins=pf*mir, vectors=yin*2, use_head=True, head_size=3., color= 0xff8c00) #yIn
    plot += k3d.vectors(origins=pf*mir, vectors=yout*2, use_head=True, head_size=3., color= 0xff8c00) #yOut
    plot += k3d.vectors(origins=pf*mir, vectors=refl*2, use_head=True, head_size=3., color=0x00ff00)

    ey = np.array([0,1,0])
    ex = np.array([1,0,0])
    for i in range(Nm):
        mirror = Box(size=(1,10,10)).mesh

        mirror.transform.custom_matrix = pad3to4(getrot(ex, refl[i])) #get rotation matrix of mirror
        mirror.transform.translation = pf*mir[i]
        plot += mirror

    return plot

In [17]:
def SixMirror(dx=27.77, dy=8.0, dz=16.685, d=4.750, dzF=1.5825, Rfast=25.0):
    

    p1 = np.array([0,0,0])
    p2 = np.array([dx, dy, dzF])
    p3 = np.array([0, dy, dzF])
    p4 = np.array([dx, 2*dy, 0])
    p5 = np.array([d, dy+d, dz])
    p6 = np.array([dx-d, dy-d, dz])
    
    ps = np.stack([p1,p2,p3,p4,p5,p6], axis=0)
    geom = geometry(ps)
    ns = geom['refl']
    hi = 12.7
    qi=7.75
    elements = [Mirror(p=p1, n=ns[0], diam=qi),\
                CurvedMirror(p=p2, n=ns[1], diam=qi, R=Rfast, curv='CC'),\
                CurvedMirror(p=p3, n=ns[2], diam=qi, R=Rfast, curv='CC'),\
                Mirror(p=p4, n=ns[3], diam=qi),\
                Mirror(p=p5, n=ns[4], diam=hi),\
                Mirror(p=p6, n=ns[5], diam=hi)]
    return elements, geom

In [18]:
def rot_vec(v, ax, ang):
    if ang==0.0:
        return v
    else:
        return v*np.cos(ang) + np.cross(ax,v)*np.sin(ang) + ax@v*ax*(1. - np.cos(ang))

rot_vec(np.array([0,0,1]), np.array([0,1,0]), np.pi/2)

array([1.000000e+00, 0.000000e+00, 6.123234e-17])

In [61]:
def propagate_system(elements, order, rays, Nrt=1):
    rs = rays.copy()
    ind = np.arange(rs.shape[1])
    allrays = [rs,]
    allind = [ind,]
    for _ in range(Nrt):
        for o in order:
            try:
                rs, ind = elements[o].propagate(rs, indices=ind)
            except:
                print('Propagation failed at element {}'.format(str(elements[o])))
                break
            else:
                allrays.append(rs)
                allind.append(ind)
        
    return allrays, allind

In [20]:
def get_trajectories(allrays, allind):
    traj = []
    for i in range(allrays[0].shape[1]):
        steps = []
        for s in range(len(allind)):
            if i in allind[s]:
                idx = np.where(i==allind[s])[0]
                steps.append(allrays[s][0,idx,:])
            else:
                break
        traj.append(steps)
    return traj

## Six mirror cavity

In [21]:
elements, geom = SixMirror()

x0 = 0.5*(geom['mir'][2] + geom['mir'][1])
n0 = norm(geom['mir'][1] - geom['mir'][2])
rs = ray_bundle(p=x0, n=n0, n_radii=5,n_angles=2, R=0.2)

rays = rs.copy()

rays[0,:,:] += x0[None,:]

order = [2,3,4,5,0,1]
screen = Screen(p=x0, n=-n0, diam=7.75)
elements.append(screen)
order.append(6)

In [22]:
allrays, allind = propagate_system(elements, order, rays)
traject = get_trajectories(allrays, allind)

In [23]:
plot = k3d.plot(camera_auto_fit=True, antialias=True)

plot += k3d.vectors(origins=x0, vectors=n0)
#plot_rays(rays, plot)
for el in elements:
    plot += el.plot(opacity=0.4)

plot += screen.plot_outline()

for t in traject:
    plot += k3d.line(t, color=0x00ff00)
plot.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [25]:
r0 = x0 + np.array([0,0.2,0.2])
s0 = n0 
ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)

rs, inds = propagate_system(elements, order, ray0)
traj = get_trajectories(rs, inds)[0]

In [143]:
def find_eigenray(elements, order, ray0, lr = 0.05, maxiter=500, tol=1e-5):
    
    rcur = ray0.copy()
    for i in range(maxiter):
        rs, inds = propagate_system(elements, order, rcur)
        rnew = rs[-1]
        res = np.max(np.abs(rcur.flatten() - rnew.flatten()))
        rcur = (1.-lr)*rcur + lr*rnew
        if res<tol:
            break
    return rcur

def find_eigenray_animated(elements, order, ray0, lr = 0.05, maxiter=500, tol=1e-5):
    
    rcur = ray0.copy()
    trajs = []
    for i in range(maxiter):
        rs, inds = propagate_system(elements, order, rcur)
        traj = get_trajectories(rs, inds)[0]
        trajs.append(traj)
        rnew = rs[-1]
        res = np.max(np.abs(rcur.flatten() - rnew.flatten()))
        rcur = (1.-lr)*rcur + lr*rnew
        if res<tol:
            break
    return rcur, trajs

In [27]:
reig, trajs = find_eigenray_animated(elements, ray0, tol=1e-6)

ts_dict = {str(t/100): trajs[t] for t in range(0, len(trajs), 10)}

In [29]:
plot_ts = k3d.plot(camera_auto_fit=True, antialias=True)


for el in elements:
    plot_ts += el.plot(opacity=0.4)

plot_ts += screen.plot_outline()
plot_ts += k3d.line(ts_dict, shader='mesh', width=0.2, color=0x00ff00)
plot_ts.display()

Output()

In [30]:
def SixMirrorPert(dx=27.77, dy=8.0, dz=16.685, d=4.750, dzF=1.5825, Rfast=25.0, eps=np.zeros((6,5))):

    p1 = np.array([0,0,0])
    p2 = np.array([dx, dy, dzF])
    p3 = np.array([0, dy, dzF])
    p4 = np.array([dx, 2*dy, 0])
    p5 = np.array([d, dy+d, dz])
    p6 = np.array([dx-d, dy-d, dz])
    
    ps = np.stack([p1,p2,p3,p4,p5,p6], axis=0)
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    #add perturbations
    ps += eps[:,:3]
    axx = geom['xin']
    axy = 0.5*(geom['yin']+geom['yout'])
    ns = [rot_vec(rot_vec(ns[i], axx[i], np.deg2rad(eps[i,3])), axy[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]
    
    hi = 12.7
    qi=7.75
    elements = [Mirror(p=ps[0], n=ns[0], diam=qi),\
                CurvedMirror(p=ps[1], n=ns[1], diam=qi, R=Rfast, curv='CC'),\
                CurvedMirror(p=ps[2], n=ns[2], diam=qi, R=Rfast, curv='CC'),\
                Mirror(p=ps[3], n=ns[3], diam=qi),\
                Mirror(p=ps[4], n=ns[4], diam=hi),\
                Mirror(p=ps[5], n=ns[5], diam=hi)]
    return elements, geom

In [31]:
#set up unperturbed system
elements, geom = SixMirrorPert()
x0 = 0.5*(geom['mir'][2] + geom['mir'][1])
n0 = norm(geom['mir'][1] - geom['mir'][2])

r0 = x0
s0 = n0 
ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)

order = [2,3,4,5,0,1]
screen = Screen(p=x0, n=-n0, diam=7.75)
elements.append(screen)
order.append(6)

In [32]:
reig0 = find_eigenray(elements, ray0, tol=1e-6)
traj0 = np.squeeze(np.stack(get_trajectories(*propagate_system(elements, order, reig0))[0]))

In [33]:
def get_deviation(eps, ray0):
    elements, _ = SixMirrorPert(eps=eps)
    elements.append(screen)
    try:
        reig = find_eigenray(elements, ray0, tol=1e-5)
        traj = np.squeeze(np.stack(get_trajectories(*propagate_system(elements, order, reig))[0]))
    except ValueError:
        traj = np.ones_like(traj0)*np.nan
    modedist = np.linalg.norm(traj - traj0, axis=1)
    return modedist

In [34]:
eps = np.zeros((6,5))

el = 1
ax = 3
es = np.linspace(-1, 1, 50)
dists = np.empty((len(es),8))
for i, e in enumerate(es):
    eps[el,ax] = e
    dists[i,:] = get_deviation(eps, ray0)

In [39]:
plt.figure()
axis=['x','y','z',r'$\theta_x$',r'$\theta_y$']
labels = ['Source', 'Mir 1', 'Mir 2 (CC)', 'Mir 3 (CC)', 'Mir 4', 'Mir 5', 'Mir 6', 'Screen']
for i in range(dists.shape[1]):
    plt.plot(es, dists[:,i], marker='.', label=labels[i])
plt.title('Moving {} in {}'.format(labels[el], axis[ax]))
plt.xlabel(axis[ax])
plt.ylabel('Deviation from eigenray [mm]')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
from itertools import product
from joblib import Parallel, delayed

In [43]:
def pert_one(el, ax, npts=10):
    eps = np.zeros((6,5))
    es = np.linspace(0, 1, npts)
    dists = np.empty((len(es),8))
    for i, e in enumerate(es):
        eps[el,ax] = e
        dists[i,:] = get_deviation(eps, ray0)
    return dists

In [63]:
all_coords = list(product(range(6),range(5)))
all_pert = Parallel(n_jobs=8)(delayed(pert_one)(i,j) for i, j in all_coords)

In [60]:
all_pert = [pert_one(i,j) for i, j in all_coords]

AttributeError: 'str' object has no attribute 'fomrat'

In [64]:
pert_arr = np.array(all_pert).reshape(6,5,10,8)

In [67]:
f, ax = plt.subplots(nrows=6, ncols=5, figsize=(5*2, 6*2), sharex=True, sharey=True)
esp = np.linspace(0, 1, 10)
labelsp = ['Mir 1', 'Mir 2 (CC)', 'Mir 3 (CC)', 'Mir 4', 'Mir 5', 'Mir 6']
for i, j in all_coords:
    ax[i,j].plot(esp, pert_arr[i,j])
    
    if i==0:
        ax[0,j].set_title(axis[j])
        
    if j==0:
        ax[i,0].text(x=-0.3, y=1, s=labelsp[i], ha='center', rotation=90)
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
def get_pert_traj(eps, ray0):
    elements, _ = SixMirrorPert(eps=eps)
    elements.append(screen)
    try:
        reig = find_eigenray(elements, ray0, tol=1e-5)
        traj = np.squeeze(np.stack(get_trajectories(*propagate_system(elements, order, reig))[0]))
        return traj
    except ValueError:
        return None

In [46]:
def pert_one(el, ax, npts=10):
    eps = np.zeros((6,5))
    es = np.linspace(0, 1, npts)
    dists = np.empty((len(es),8))
    for i, e in enumerate(es):
        eps[el,ax] = e
        dists[i,:] = get_deviation(eps, ray0)
    return dists

## Poincaree hitpattern

In [47]:
reig0 = find_eigenray(elements, ray0, tol=1e-6)

In [48]:
reigp = reig0.copy()
reigp[1,0,2] += 0.1

In [68]:
reigp = reig0.copy()
reigp[0,0,2] += 0.1

In [69]:
traj_hit = np.squeeze(np.stack(get_trajectories(*propagate_system(elements, order, reigp, Nrt=100))))
traj_hit.shape

(701, 3)

In [70]:
hit_scr = traj_hit[7::7,:]
hit_idx = np.arange(hit_scr.shape[0])

plt.figure()
plt.scatter(hit_scr[:,1], hit_scr[:,2], c=hit_idx, cmap='jet')
plt.title('Poincare hit pattern (screen)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close('all')

In [51]:
plot_pc = k3d.plot(camera_auto_fit=True, antialias=True)


for el in elements:
    plot_pc += el.plot(opacity=0.4)

plot_pc += k3d.line(traj_hit, width=0.2, color=0x00ff00)
plot_pc.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

# Lens cavity

In [52]:
def LenscavPert(arm1=50., arm2=55., base=19., angle=np.deg2rad(150), lens_dist=11.075, lens_diam=6.35, lens_thick=4., Rlens=5.0, eps=np.zeros((8,5))):
    
    p0 = np.array([lens_dist/2.-lens_thick/2.,0,0])
    p1 = np.array([lens_dist/2.+lens_thick/2.,0,0])
    p2 = np.array([arm1/2.,0,0])
    p3 = np.array([np.cos(angle)*arm2/2.,base,np.sin(angle)*arm2/2.])
    p4 = np.array([-np.cos(angle)*arm2/2.,base,-np.sin(angle)*arm2/2.])
    p5 = np.array([-arm1/2.,0,0])
    p6 = np.array([-lens_dist/2.-lens_thick/2.,0,0])
    p7 = np.array([-lens_dist/2.+lens_thick/2.,0,0])

    
    ps = np.stack([p0,p1,p2,p3,p4,p5,p6,p7], axis=0)
    geom = geometry(ps)
    ns = geom['refl']
    ns[0,:] = np.array([-1,0,0])
    ns[1,:] = np.array([-1,0,0])
    ns[6,:] = np.array([-1,0,0])
    ns[7,:] = np.array([-1,0,0])
    ps = geom['mir']
    #add perturbations
    ps += eps[:,:3]
    axx = geom['xin']
    axy = 0.5*(geom['yin']+geom['yout'])
    ns = [rot_vec(rot_vec(ns[i], axx[i], np.deg2rad(eps[i,3])), axy[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]
    
    hi = 12.7
    qi=7.75
    ng = 1.41
    elements = [Glass(p=ps[0], n=ns[0], diam=lens_diam, n2=ng),\
                CurvedGlass(p=ps[1], n=ns[1], diam=lens_diam, R=Rlens, curv='CC', n1=ng),\
                Mirror(p=ps[2], n=ns[2], diam=qi),\
                Mirror(p=ps[3], n=ns[3], diam=qi),\
                Mirror(p=ps[4], n=ns[4], diam=hi),\
                Mirror(p=ps[5], n=ns[5], diam=hi),\
                CurvedGlass(p=ps[6], n=ns[6], diam=lens_diam, R=Rlens, curv='CX', n2=ng),\
                Glass(p=ps[7], n=ns[7], diam=lens_diam, n1=ng),\
               ]
    return elements, geom

In [91]:
LensElements, LensGeom = LenscavPert(angle=np.deg2rad(30))

In [ ]:
for el in LensElements:
    print(el, el.p, el.n)

In [ ]:
plot_geometry(LensGeom)

In [77]:
plot_lc = k3d.plot(camera_auto_fit=True, antialias=True)

for el in LensElements:
    plot_lc += el.plot(opacity=0.4)

plot_lc += k3d.line(LensTraj, width=0.2, color=0x00ff00)
#plot_rays(Lens_ray0, plot_lc)
#r2, _ = LensElements[0].propagate(Lens_ray0)
#plot_rays(r2, plot_lc)
#r3, _ = LensElements[1].propagate(r2)
#plot_rays(r3, plot_lc)

plot_lc.display()

Output()

In [92]:
Lens_x0 = 0.5*(LensGeom['mir'][0] + LensGeom['mir'][7])
Lens_n0 = norm(LensGeom['mir'][0] - LensGeom['mir'][7])

#Lens_ray0 = np.stack([np.atleast_2d(Lens_x0), np.atleast_2d(Lens_n0)], axis=0)
Lens_ray0 = ray_bundle(n=Lens_n0, n_radii=3,n_angles=2, R=0.2)
Lens_order = [0,1,2,3,4,5,6,7]
Lens_screen = Screen(p=Lens_x0, n=Lens_n0, diam=7.75)
LensElements.append(Lens_screen)
Lens_order.append(8)

In [93]:
LensTraj = np.squeeze(np.stack(get_trajectories(*propagate_system(LensElements, Lens_order, Lens_ray0, Nrt=1))))
LensTraj.shape

(19, 10, 3)

In [94]:
LensTraj1 = np.squeeze(np.stack(get_trajectories(*propagate_system(LensElements, Lens_order, Lens_ray1, Nrt=3))))
LensTraj1.shape

(28, 3)

In [149]:
plot_lc2 = k3d.plot(camera_auto_fit=True, antialias=True)

for el in LensElements:
    plot_lc2 += el.plot(opacity=0.4)

#plot_lc2 += k3d.line(LensTraj1, width=0.2, color=0x00ff00)
plot_lc2 += k3d.line(Lens_reig_trajs_dict, width=0.5, color=0x00ff00)
Lens_reig_trajs_dict
plot_lc2.display()

Output()

In [163]:
len(Lens_reig_trajs)

1

In [146]:
Lens_ray1 = np.stack([np.atleast_2d(Lens_x0), np.atleast_2d(Lens_n0)], axis=0)
Lens_reig = find_eigenray(LensElements, Lens_order, Lens_ray1, tol=1e-4, lr=0.05, maxiter=500)

In [162]:
Lens_reig, Lens_reig_trajs = find_eigenray_animated(LensElements, Lens_order, Lens_ray1, tol=1e-6, lr=0.05, maxiter=200)

Lens_reig_trajs_dict = {str(t/100): Lens_reig_trajs[t] for t in range(0, len(Lens_reig_trajs), 1)}

In [164]:
Lens_rayp = Lens_ray1.copy()
Lens_rayp[0,0,2] += 1e-4

In [169]:
LensTrajp = np.squeeze(np.stack(get_trajectories(*propagate_system(LensElements, Lens_order, Lens_rayp, Nrt=6))))
LensTrajp.shape

Propagation failed at element <__main__.CurvedGlass object at 0x0000018096336F48>
Propagation failed at element <__main__.CurvedGlass object at 0x0000018096336708>


(40, 3)

In [168]:
hit_lens = LensTrajp[9::9,:]
hit_idxl = np.arange(hit_lens.shape[0])

plt.figure()
plt.scatter(hit_lens[:,1], hit_lens[:,2], c=hit_idxl, cmap='jet')
plt.title('Poincare hit pattern (screen)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [140]:
plot_lc3 = k3d.plot(camera_auto_fit=True, antialias=True)

for el in LensElements:
    plot_lc3 += el.plot(opacity=0.4)

plot_lc3 += k3d.line(LensTrajp, width=0.2, color=0x00ff00)

plot_lc3.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [170]:
%load_ext line_profiler

In [175]:
%lprun -f Mirror.intersect_all find_eigenray(LensElements, Lens_order, Lens_ray1, tol=1e-4, lr=0.05, maxiter=500)

Timer unit: 1e-07 s

Total time: 0.0007553 s
File: <ipython-input-6-b99d583a3499>
Function: intersect_all at line 32

Line #      Hits         Time  Per Hit   % Time  Line Contents
    32                                               def intersect_all(self, ray, return_msk=False):
    33                                                   #shape of Ray [2xNrays]
    34         7        205.0     29.3      2.7          r, s = ray
    35         7        651.0     93.0      8.6          sn = s@self.n
    36                                           
    37         7        557.0     79.6      7.4          t = ((self.p - r)@self.n)/sn
    38         7        923.0    131.9     12.2          msk = np.abs(sn)>np.finfo(np.float32).eps
    39         7       5132.0    733.1     67.9          x = r[msk] + t[msk,None]*s[msk]
    40         7         52.0      7.4      0.7          if return_msk:
    41         7         33.0      4.7      0.4              return x, msk
    42                     